<a href="https://colab.research.google.com/github/boothmanrylan/MSSForestDisturbances/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# imports and installs

In [ ]:
# standard python imports
import os

In [ ]:
# earth engine api import and authentication
import ee
ee.Authenticate()
ee.Initialize()

In [ ]:
# pip installable modules

try:
    import geemap.foliumap as geemap
except:
    !pip install geemap
    import geemap.foliumap as geemap

In [ ]:
# autoreload all python modules before executing code block
%load_ext autoreload
%autoreload 2

In [ ]:
# install and import my packages from github
from google.colab import drive, auth

drive.mount('/content/drive/')
%cd /content/drive/My\ Drive/colaboratory

# auth.authenticate_user()

# install msslib
! if [ ! -d ./msslib ]; then git clone https://github.com/boothmanrylan/msslib.git; fi
%cd msslib
! git pull
try:
    from msslib import msslib
except:
    ! pip install .
    from msslib import msslib
%cd ../

# install sequencer
! if [ ! -d ./sequencer ]; then git clone https://github.com/boothmanrylan/sequencer.git; fi
%cd sequencer
! git pull
try:
    import sequencer
except:
    ! pip install .
    import sequencer
%cd ../

# install MSSForestDisturbances
! if [ ! -d ./MSSForestDisturbances ]; then git clone https://github.com/boothmanrylan/MSSForestDisturbances.git; fi
%cd MSSForestDisturbances
! git pull
try:
    import mfd
except:
    ! pip install .
    import mfd
%cd ../

# Reload

Run these blocks if the modules have been updated since the runtime was started

In [ ]:
# reload msslib
%cd ./msslib
! git pull
%cd ../

In [ ]:
# reload sequencer
%cd ./sequencer/
! git pull
%cd ../

In [ ]:
# reload MSSForestDisturbances
%cd ./MSSForestDisturbances/
! git pull
%cd ../

# Test

In [ ]:
def get_valid_burns(events):
    years = events.distinct("year").aggregate_array("year")

    def _valid(year):
        curr_events = events.filter(ee.Filter.eq("year", year))
        exclusively_burnt = curr_events.reduce(ee.Reducer.allNonZero())
        burnt_often = curr_events.reduce(ee.Reducer.sum()).gt(1)
        return exclusively_burnt.Or(burnt_often).selfMask()

    result = ee.ImageCollection.fromImages(years.map(_valid))
    return result.reduce(ee.Reducer.anyNonZero())

def postprocess(event):
    event = mfd.postprocess.postprocess(event)
    event = mfd.postprocess.squash_extra_classes(event)
    event = mfd.postprocess.set_true_date(event)
    return event

In [ ]:
grid_split_qc = ee.FeatureCollection('users/boothmanrylan/gridSplitForestedQC')

grid_cell = grid_split_qc.filter(ee.Filter.eq("id", 48)).first().geometry()


In [ ]:
def set_true_date(im):
    date = im.get('DATE_ACQUIRED')
    millis = ee.Date.parse('YYYY-MM-dd', date).millis()
    return im.set('system:time_start', millis)

In [ ]:
quebec_fires = ee.ImageCollection("users/boothmanrylan/MSSForestDisturbances/test2")
aoi = ee.Geometry.Point([-76.90250107516832, 51.526388298517745])

cell = grid_split_qc.filterBounds(aoi)

events = mfd.postprocess.filter_and_clip(quebec_fires, cell)

processed_events = events.map(mfd.postprocess.postprocess)
processed_events = events.map(mfd.postprocess.squash_extra_classes)
# print(processed_events.first().getInfo())
# processed_events = events.map(set_true_date)
# print(processed_events.first().getInfo())

events_carried_forward = mfd.postprocess.carry_observations_forward(processed_events)
burns = sequencer.match_sequence(events_carried_forward, ee.List([1, 1, 1]), 0)
burns = mfd.postprocess.get_year_from_index(processed_events, burns)

burns2 = sequencer.find_stable_sequence(processed_events.toBands(), 1, 3, 10, 0)
burns2 = mfd.postprocess.get_year_from_index(processed_events, burns2)

permanent_burns = mfd.postprocess.get_burn_year_spread(processed_events).gt(5)

valid_burns = get_valid_burns(processed_events).And(permanent_burns.Not())
all_burns = processed_events.reduce(ee.Reducer.anyNonZero())

In [ ]:
cell = grid_split_qc.filterBounds(aoi)
Map = geemap.Map()
Map.addLayer(cell)
Map

In [ ]:
binary_vis = {'min': 0, 'max': 1}

Map = geemap.Map()
# Map.addLayer(grid_split_qc, {}, 'Grid Split QC')
# Map.addLayer(grid_cell, {}, 'Grid Cell of Interest')
Map.addLayer(valid_burns, binary_vis, 'Valid Burns')
Map.addLayer(all_burns, binary_vis, 'All Burns')
Map.addLayer(burns, {'min': 1972, 'max': 1984, 'palette': mfd.utils.RAINBOW_PALETTE}, "Exact Burns")
Map.addLayer(burns2, {'min': 1972, 'max': 1984, 'palette': mfd.utils.RAINBOW_PALETTE}, "Stable Burns")
Map.centerObject(aoi, 10)

Map

In [ ]:
ee.EE